<a href="https://colab.research.google.com/github/Knowsoeun/AI_Study/blob/main/ResNet%EA%B5%AC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*더 필요한 것들

-Batch Normalization 등의 Layer  
-CNN을 다르게 구성해야 한다.  
　-> Sequential()은 Residual Path 구조를 기술하지 못한다.

In [ ]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import os

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_resnet20.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

input_shape = x_train.shape[1:]

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# subtract pixel mean
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 90:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 40:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
inputs = Input(shape=input_shape)
print('x shape:', inputs.shape)
conv = Conv2D(filters=16, kernel_size=3, strides=1, padding='same', # 합성곱층을 하나 설정해준다.
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
x = conv(inputs) # 합성곱층에 입력을 넣는다.
print('x shape:', x.shape) 
x = BatchNormalization()(x) #합성곱층을 거진 뒤 BatchNormalization을 거친다.

In [ ]:
x = Activation('relu')(x)	  	#---|
# Block 1_1					        	#   |
conv = Conv2D(filters=16, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)					        	#   |
y = BatchNormalization()(y)		#	  |
y = Activation('relu')(y)	  	#	  |
							               	#	  |	Residual path
conv = Conv2D(filters=16, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)					        	# 	|
y = BatchNormalization()(y)		#	  |
							              	#	  |
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)	   	#---|


* Residual path 9개  
* 실습 시간을 줄이기 위해 6개의 블럭은 주석처리함

In [ ]:
# Block 1_2					        	#   |
conv = Conv2D(filters=16, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)				        		#	  |
y = BatchNormalization()(y)		#  	|
y = Activation('relu')(y)	   	#	  |
							               	#   |	Residual path
conv = Conv2D(filters=16, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)					        	#	  |
y = BatchNormalization()(y)		#	  |
						               		#	  |
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|
# Block 1_2						#   |
conv = Conv2D(filters=16, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)						#	|
y = BatchNormalization()(y)		#	|
y = Activation('relu')(y)		#	|
								#	|	Residual path
conv = Conv2D(filters=16, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)						#	|
y = BatchNormalization()(y)		#	|
								#	|
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|
# Block 2_1						#   |
conv = Conv2D(filters=32, kernel_size=3, strides=2, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)						#	|
y = BatchNormalization()(y)		#	|
y = Activation('relu')(y)		#	|
								#	|	Residual path
conv = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)						#	|
y = BatchNormalization()(y)		#	|
								#	|
conv = Conv2D(filters=32, kernel_size=1, strides=2, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
x = conv(x)						#	|
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|
# Block 2_2						#   |
conv = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)						#	|
y = BatchNormalization()(y)		#	|
y = Activation('relu')(y)		#	|
								#	|	Residual path
conv = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)						#	|
y = BatchNormalization()(y)		#	|
								#	|
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|
# Block 2_3						#   |
conv = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)						#	|
y = BatchNormalization()(y)		#	|
y = Activation('relu')(y)		#	|
								#	|	Residual path
conv = Conv2D(filters=32, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)						#	|
y = BatchNormalization()(y)		#	|
								#	|
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|
# Block 3_1						#   |
conv = Conv2D(filters=64, kernel_size=3, strides=2, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)						#	|
y = BatchNormalization()(y)		#	|
y = Activation('relu')(y)		#	|
								#	|	Residual path
conv = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)						#	|
y = BatchNormalization()(y)		#	|
								#	|
conv = Conv2D(filters=64, kernel_size=1, strides=2, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
x = conv(x)						#	|
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|
# Block 3_2						#   |
conv = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)						#	|
y = BatchNormalization()(y)		#	|
y = Activation('relu')(y)		#	|
								#	|	Residual path
conv = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)						#	|
y = BatchNormalization()(y)		#	|
								#	|
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|
# Block 3_3						#   |
conv = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(x)						#	|
y = BatchNormalization()(y)		#	|
y = Activation('relu')(y)		#	|
								#	|	Residual path
conv = Conv2D(filters=64, kernel_size=3, strides=1, padding='same',
				kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
y = conv(y)						#	|
y = BatchNormalization()(y)		#	|
								#	|
x = keras.layers.add([x, y])	#<--|
x = Activation('relu')(x)		#---|

In [ ]:
x = AveragePooling2D(pool_size=8)(x)
y = Flatten()(x)
outputs = Dense(num_classes,
				activation='softmax',
				kernel_initializer='he_normal')(y)

model = Model(inputs=inputs, outputs=outputs)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

* Callbacks : 하나의 에폭이 끝날 때마다 해야 할 작업을 저장해 놓은 것  
- 학습하는 도중에 저장하기 -> 각 에폭이 끝난 후, 이전보다 나아졌다면 저장하기  


In [ ]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_model.{epoch:03d}.h5'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name) # 어디에 저장할지를 지정한다.

In [ ]:
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath, # ModelCheckpoint로 저장 방법을 기술하고 이것을 체크포인트에 저장한다.
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpoint, lr_scheduler]

if not data_augmentation:
	print('Not using data augmentation.')
	history = model.fit(x_train, y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  validation_data=(x_test, y_test),
			  shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Save model and weights
#if not os.path.isdir(save_dir):
#	os.makedirs(save_dir)
#model_path = os.path.join(save_dir, model_name)
#model.save(model_path)
#print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

*3개의 Residual Block를 사용하면    
-GPU 1060 : 16초/에폭  
-No GPU : 304초/에폭

*9개의 Residual Block을 모두 사용하면  
-GPU 1060: 28초/에폭  
-No GPU: 756초/에폭  